### Regularization with Dropout

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.constraints import maxnorm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

Using TensorFlow backend.


In [2]:
df = pd.read_csv('~/Documents/Deep_Learning_with_Keras/data/sonar.csv', header=None)
X = df[df.columns[:-1]].values
y = df[df.columns[-1]].values

In [3]:
label = LabelEncoder()
y = label.fit_transform(y)

In [4]:
def wider_model():
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu' ))
    model.add(Dense(30, kernel_initializer='normal', activation='relu' ))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [5]:
np.random.seed(7)
model = KerasClassifier(build_fn=wider_model, epochs=300, batch_size=16, verbose=0)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('cls', model)
])
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(results.mean(), results.std())

0.860411255721 0.0457836415776


In [6]:
def dropout_model():
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60,  kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [7]:
np.random.seed(7)
model = KerasClassifier(build_fn=dropout_model, epochs=300, batch_size=16, verbose=0)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('reg', model)
])
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(results.mean(), results.std())

0.86521645081 0.0601376478922
